In [92]:
# -*- coding: utf-8 -*-

from __future__ import division, unicode_literals

import pandas as pd
import numpy as np

import dateutil
import datetime
import time
from time import sleep

import re
import html5lib

import pickle
import json

In [93]:
# Selenium imports
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from pyvirtualdisplay import Display

import os

In [94]:
# timing function
def timefunc(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print(f.__name__, 'took', end - start, 'seconds')
        return result
    return f_timer

## Functions to scrape petition URLs

In [95]:
# Setting up chromedriver
chromedriver = 'C:/WebDriver/bin/'
os.environ['webdriver.chrome.driver'] = chromedriver
driver = webdriver.Chrome(executable_path='C:/WebDriver/bin/chromedriver.exe')

In [96]:
# go straight to most-recent pages
petitions_url = 'https://www.change.org/petitions?selected=popular_weekly' # up to 5084
driver.get(petitions_url)


def get_petition_urls(start):
    d = []
    petitions = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".col-xs-12.col-md-8.col-md-offset-2 [href]")))
    stop = start+10
    for p in petitions[start:stop]:
        d.append(p.get_attribute('href'))
    return d


def click_load_more():
    driver.find_element_by_css_selector('.btn.btn-big.btn-full.bg-brighter').click()

@timefunc
def open_list(limit):
    for i in range(0, limit):
        click_load_more()
        sleep(0.1)

def update_next10(dic, start):
    d = get_petition_urls(start)
    dic.extend(d)
    #return dic

@timefunc
def get1000(dic, limit):
    start = 4
    for i in range(0, limit):
        update_next10(dic, start)
        start += 10
    return start
"""
@timefunc
def chunk_petition_urls(dic, base, first, last, chunk):
    for c in range(first, last+1, chunk):
        d = get_petition_urls(base, c, c+chunk)
        dic.update(d)
        filename = 'master' + str(c+chunk) + '.pkl'
        with open(filename, 'wb') as f:
            pk.dump(dic,f,-1)
    return dic
"""

"\n@timefunc\ndef chunk_petition_urls(dic, base, first, last, chunk):\n    for c in range(first, last+1, chunk):\n        d = get_petition_urls(base, c, c+chunk)\n        dic.update(d)\n        filename = 'master' + str(c+chunk) + '.pkl'\n        with open(filename, 'wb') as f:\n            pk.dump(dic,f,-1)\n    return dic\n"

## Creating master dictionary of petition URLs

In [97]:
#urls1000 = chunk_petition_urls(master, petitions_url, 1, 1000, 100)
urls1000 = []
limit = 30
open_list(limit)
get1000(urls1000, limit)


#with open("pickles/urls1000.pickle", 'wb') as f:
#    pickle.dump(urls1000, f)

open_list took 7.1432271003723145 seconds
get1000 took 1.2056236267089844 seconds


304

In [98]:
print(urls1000)
len(urls1000)

['https://www.change.org/p/wrongly-convicted-savedustinjhiggs-com-help-me-change-the-narrative?source_location=petitions_browse', 'https://www.change.org/p/j-b-pritzker-governor-pritzker-veto-illinois-hb-3653?source_location=petitions_browse', 'https://www.change.org/p/cuyahoga-county-prosecutor-justice4jayla?source_location=petitions_browse', 'https://www.change.org/p/senator-ted-cruz-texans-demand-the-resignation-of-senator-ted-cruz?source_location=petitions_browse', 'https://www.change.org/p/help-save-dustin-john-higgs?source_location=petitions_browse', 'https://www.change.org/p/sen-josh-hawley-constituents-demanding-sen-josh-hawley-s-resignation?source_location=petitions_browse', 'https://www.change.org/p/impeached-twice-it-s-time-to-remove-45?source_location=petitions_browse', 'https://www.change.org/p/president-joe-biden-award-uscp-officer-eugene-goodman-the-medal-of-freedom?source_location=petitions_browse', 'https://www.change.org/p/voters-force-joel-osteen-to-pay-back-any-and-

90

# access each url and visit 
Make pandas dataframe that has petition name, image link, description, date<br/>
Try to put into json file with structure:<br/>

"petitions": \[<br/>
    &emsp;&emsp;{<br/>
        &emsp;&emsp;&emsp;"title" : "",<br/>
        &emsp;&emsp;&emsp;"description" : "",<br/>
        &emsp;&emsp;&emsp;"image link" : ""<br/>
    &emsp;&emsp;}, ...<br/>
\]<br/>

In [90]:
#with open("pickles/urls1000.pickle", 'wb') as f:
#    pickle.dump(urls1000, f)

In [101]:
json_dictionary = {"petitions":[]}

for url in urls1000:   

    # visit url:
    driver.get(url)

    # collect title -- note: slow reloads can cause blank page-- sleep(0.1) to avoid but still continue if not
    try:
        title = driver.find_element_by_css_selector(".mtl.mbxxxl.xs-mts.xs-mbxs.petition-title").get_attribute("innerHTML")
    except NoSuchElementException:
        continue

    # collect image url

    image_url = driver.find_element_by_css_selector(".sc-fzpmMD.fPcnhl").get_attribute("src")

    # collect description from <p> tags
    description = ""
    p_tags = driver.find_elements_by_css_selector(".mbl.type-break-word.type-l.rte [innerHTML]")
    for p in p_tags:
        description += p.get_attribute("innerHTML")

    # define dictionary of petition data
    petition_dictionary = {}
    petition_dictionary["title"] = title
    petition_dictionary["description"] = description
    petition_dictionary["image_url"] = image_url

    json_dictionary["petitions"].append(petition_dictionary)
    sleep(0.1)

with open("petitions.json", "w") as outfile:
    json.dump(json_dictionary, outfile)
  
